In [1]:
%pip install selenium
%pip install lxml
%pip install --upgrade --quiet  markdownify

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from selenium import webdriver
from lxml import etree
from lxml import html 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup

import pandas as pd

In [3]:
chrome_options = Options()
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox") # linux only
chrome_options.add_argument("--headless=new") # for Chrome >= 109
# chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works

DRIVER = webdriver.Chrome(options=chrome_options)

# Acquire List of Nobel Laureates

In [6]:
BASE_URL = "https://en.wikipedia.org/wiki/List_of_Nobel_laureates"

In [7]:
def get_etree_from(driver, URL):
    driver.get(URL)
    tree = etree.HTML(driver.page_source)
    
    return tree

In [8]:
tree = get_etree_from(DRIVER, BASE_URL)

In [90]:
## INITIALISE PANDAS DATAFRAME
columns = ["Year", "Physics", "Chemistry", "Physiology/Medicine", "Literature", "Peace", "Economic Sciences"]
df = pd.DataFrame(columns=columns)

## EXTRACT TABLE CONTAINING ALL NOBEL LAUREATES FROM ETREE
table = tree.xpath("//table[contains(@class, 'wikitable sortable sticky-header jquery-tablesorter')]")
table_body = table[0].xpath("./tbody/*") # i dont know why i cant join this xpath query to the one above im so confused

## PROCESS THE TABLE BODY! cleaning is here too
for index, item in enumerate(table_body):
    print(f"Processing row {index+1}...")
    processing_row = item.xpath("*")
    df_row = []

    # CLEAN VALUE TO INGEST
    # TODO: capture URL of authors
    for table_value in processing_row:
        value = ''.join(table_value.itertext())
        value = value.split("\n")[0]
        value = value.split("[b]")[0]
        df_row.append(value.strip())
        
    if (len(df_row)) == 6: #lacking economic science
        df_row.append("-")
    elif (len(df_row) < 6): #something is fucked (probably world war 2)
        while (len(df_row) <= 6):
            df_row.append("No Record")

    df.loc[-1] = df_row
    df.index = df.index + 1  # shifting index
    df = df.sort_index()  # sorting by index

Processing row 1...
Processing row 2...
Processing row 3...
Processing row 4...
Processing row 5...
Processing row 6...
Processing row 7...
Processing row 8...
Processing row 9...
Processing row 10...
Processing row 11...
Processing row 12...
Processing row 13...
Processing row 14...
Processing row 15...
Processing row 16...
Processing row 17...
Processing row 18...
Processing row 19...
Processing row 20...
Processing row 21...
Processing row 22...
Processing row 23...
Processing row 24...
Processing row 25...
Processing row 26...
Processing row 27...
Processing row 28...
Processing row 29...
Processing row 30...
Processing row 31...
Processing row 32...
Processing row 33...
Processing row 34...
Processing row 35...
Processing row 36...
Processing row 37...
Processing row 38...
Processing row 39...
Processing row 40...
Processing row 41...
Processing row 42...
Processing row 43...
Processing row 44...
Processing row 45...
Processing row 46...
Processing row 47...
Processing row 48...
P

# Further Cleaning

In [91]:
print(df)
df.to_csv("Nobel_Laureates.csv")

     Year                                         Physics  \
0    2024                   John Hopfield;Geoffrey Hinton   
1    2023   Pierre Agostini;Ferenc Krausz;Anne L'Huillier   
2    2022       Alain Aspect;John Clauser;Anton Zeilinger   
3    2021  Giorgio Parisi;Klaus Hasselmann;Syukuro Manabe   
4    2020    Roger Penrose;Reinhard Genzel;Andrea M. Ghez   
..    ...                                             ...   
119  1905                                  Philipp Lenard   
120  1904                                   Lord Rayleigh   
121  1903        Henri Becquerel;Pierre Curie;Marie Curie   
122  1902                   Hendrik Lorentz;Pieter Zeeman   
123  1901                                 Wilhelm Röntgen   

                                             Chemistry  \
0            David Baker;Demis Hassabis;John M. Jumper   
1           Moungi Bawendi;Louis E. Brus;Alexey Ekimov   
2    Carolyn Bertozzi;Morten P. Meldal;Karl Barry S...   
3                        Benjamin L